In [62]:
import numpy as np
import scipy.io as sio  #for loading matlab data
import datetime #for loading matlab data
import os #for loading matlab data
from scipy import stats
import pandas as pd

In [72]:
#problem 1(a)

filePath = 'C:\\Users\\Roxanne\\Documents\\GitHub\\PhD-class\\MATH-584\\Hwk4\\Nov-2014\\'
namelist = []
for i,j,k in os.walk(filePath):
    namelist.append(k)

namelist = namelist[0]

#download the data
begin = datetime.date(2014,11,3)
end = datetime.date(2014,11,26)

#         t.append((np.array((LOB['EventTime'][0][0][:,0]))-3600000*9.5)*1e-3) #time in seconds, measured from NASDAQ opening at 9:30am
#         bid = np.vstack((bid,np.array(LOB['BuyPrice'][0][0]*1e-4)))
#         bidvol = np.vstack((bidvol,np.array(LOB['BuyVolume'][0][0]*1.0)))
#         ask = np.vstack((ask,np.array(LOB['SellPrice'][0][0]*1e-4)))
#         askvol = np.vstack((askvol,np.array(LOB['SellVolume'][0][0]*1.0)))
#         nMO = np.vstack((nMO,np.array(LOB['NumberMO'][0][0]*1.0))) #total number of MOs submitted in the given 0.1 second interval
#         vMO = np.vstack((vMO,np.array(LOB['VolumeMO'][0][0]*1.0))) #total volume of MOs submitted in the given 0.1 second interval
#         MO = np.vstack((MO,np.array(LOB['MO'][0][0]*1.0))) #information about market orders

lam = [] # this is our estimate lambda, for every day, every ticker
tickers = ['MSFT','GOOG']

for ticker in tickers:
    date = [] # used when saving data to csv file
    for i in range((end - begin).days+1):
        day = begin + datetime.timedelta(days=i)
        mat_data_name = str(ticker)+'_'+str(day)[:4]+str(day)[5:7]+str(day)[8:10]+'.mat'
        if mat_data_name in namelist:
            date.append(str(day))
            data = sio.loadmat(filePath+mat_data_name)
            LOB = data['LOB']
            
            bid = np.array(LOB['BuyPrice'][0][0]*1e-4)
            ask = np.array(LOB['SellPrice'][0][0]*1e-4)
            vMO = np.array(LOB['VolumeMO'][0][0]*1.0) #total volume of MOs submitted in the given 0.1 second interval
            
            
            #construct linear regression
            time_frequency = 10 #reduce frequency by times
            
            midprice = (ask[:,0] + bid[:,0]) / 2
            midprice = midprice[::time_frequency]
            
            orderflow = np.cumsum(vMO[:,0]-vMO[:,1])
            orderflow = orderflow[::time_frequency]
            
            #take difference
            diff_midprice = np.diff(midprice)
            diff_orderflow = np.diff(orderflow)
            
            # drop off the mean effect
            diff_midprice = diff_midprice -  np.mean(diff_midprice)
            diff_orderflow = diff_orderflow - np.mean(diff_orderflow)
            
            slope, intercept, r_value, p_value, std_err = stats.linregress(diff_orderflow,diff_midprice)
            lam.append(slope)

In [74]:
# saving the results
lam = np.array(lam)
lam = np.reshape(lam,(len(tickers),-1)).T
lam = pd.DataFrame(lam,columns = tickers,index = date)
lam.to_csv('lambda_question1a.csv')

In [ ]:
#problem 1(b)

lam = [] # this is our estimate lambda, for every day, every ticker

for ticker in tickers:
    for i in range((end - begin).days+1):
        day = begin + datetime.timedelta(days=i)
        mat_data_name = str(ticker)+'_'+str(day)[:4]+str(day)[5:7]+str(day)[8:10]+'.mat'
        if mat_data_name in namelist:
            data = sio.loadmat(filePath+mat_data_name)
            LOB = data['LOB']
            
            t = (np.array((LOB['EventTime'][0][0][:,0]))-3600000*9.5)*1e-3 #time in seconds, measured from NASDAQ opening at 9:30am
            bid = np.array(LOB['BuyPrice'][0][0]*1e-4)
            bidvol = np.array(LOB['BuyVolume'][0][0]*1.0)
            ask = np.array(LOB['SellPrice'][0][0]*1e-4)
            askvol = np.array(LOB['SellVolume'][0][0]*1.0)
            nMO = np.array(LOB['NumberMO'][0][0]*1.0) #total number of MOs submitted in the given 0.1 second interval
            vMO = np.array(LOB['VolumeMO'][0][0]*1.0) #total volume of MOs submitted in the given 0.1 second interval
            MO=np.array(LOB['MO'][0][0]*1.0) #information about market orders
            
            #construct linear regression
            time_frequency = 10 #reduce frequency by times
            
            midprice = (ask[:,0] + bid[:,0]) / 2
            midprice = midprice[::time_frequency]
            
            orderflow = np.cumsum(vMO[:,0]-vMO[:,1])
            orderflow = orderflow[::time_frequency]
            
            #take difference
            diff_midprice = np.diff(midprice)
            diff_orderflow = np.diff(orderflow)
            
            # drop off the mean effect
            diff_midprice = diff_midprice -  np.mean(diff_midprice)
            diff_orderflow = diff_orderflow - np.mean(diff_orderflow)
            
            slope, intercept, r_value, p_value, std_err = stats.linregress(diff_orderflow,diff_midprice)
            lam.append(slope)